## Construct Dataset

### Download Videos from Youtube 

Construct a folder named Dataset in your working directory to store the whole dataset. Construct a new folder named Videos inside it where you can download the videos.
You need to install few dependencies like youtube-dl and ffmpeg.


`sudo apt-get install ffmpeg
 sudo pip install youtube-dl
`

In [ ]:
from __future__ import unicode_literals
import csv 
import sys
import subprocess as sp
import youtube_dl
import platform
import os


if platform.system() == 'Linux':
    FFMPEG_BIN = "ffmpeg" # on Linux 

if platform.system() == 'Windows':
    FFMPEG_BIN = "ffmpeg.exe" # on windows


ydl = youtube_dl.YoutubeDL({'outtmpl': '%(id)s%(ext)s'})
 

# Change this to the where you placed the video dataset file    
CSV_FILE = "./Video-Dataset.csv"
    
f = open(CSV_FILE, 'r')
dataset_folder = "Dataset" + "/" + "Videos"

reader = csv.reader(f)

for row in reader:
    category = str(row[0]).rstrip()
    link = str(row[1]).rstrip()
    time = str(row[2]).rstrip()
    dur = str(row[3]).rstrip()
    
    vid_name = link.split('=')[1].replace('-','_') + ".mp4"
    
    directory = dataset_folder + "/" + category 
    
    if not os.path.exists(directory):
        os.makedirs(directory) 
       
    with ydl:
        result = ydl.extract_info(link,download=False)
        
    if 'entries' in result:
       # Can be a playlist or a list of videos
        video = result['entries'][0]
    else:
    # Just a video
        video = result
  
    
    if os.path.exists(directory + "/" + vid_name) == False:
        
        formats = video['formats']
        formats = [x for x in formats if x['vcodec'] != 'none']
        formats = sorted(formats, key=lambda k: k['width'], reverse=True)
        
        for _format in formats:
            if _format['vcodec'] != 'none':
                url = _format['url']
                command = [ FFMPEG_BIN,
                            '-ss',time,
                            '-t',dur,
                            '-i', url, 
                             directory + "/" + vid_name ]
                ffmpeg = sp.Popen(command, stderr=sp.STDOUT,stdout = sp.PIPE)
                out, err = ffmpeg.communicate()
                if os.path.exists(directory + "/" + vid_name):
                    break;


### Extract Keyframes from Videos
In this step, we extract keyframes from each video using the technique of sum of absolute differences.For this you need to install opencv. One easy way to install opencv is using pip as :

`sudo pip install opencv-python
`
Construct a new directory inside Dataset called Keyframes to store the key frames for each video

In [9]:
import cv2
import operator
import numpy as np

import matplotlib
matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!

import matplotlib.pyplot as plt
import sys
from scipy.signal import argrelextrema


#Setting fixed threshold criteria
USE_THRESH = False
#fixed threshold value
THRESH = 0.6
#Setting fixed threshold criteria
USE_TOP_ORDER = True
#Setting local maxima criteria
USE_LOCAL_MAXIMA = False
#Number of top sorted frames
NUM_TOP_FRAMES = 7




def smooth(x, window_len=13, window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.
    output:
        the smoothed signal
        
    example:
    import numpy as np    
    t = np.linspace(-2,2,0.1)
    x = np.sin(t)+np.random.randn(len(t))*0.1
    y = smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter 
    """
    print(len(x), window_len)
    if x.ndim != 1:
        raise ValueError, "smooth only accepts 1 dimension arrays."

    if x.size < window_len:
        raise ValueError, "Input vector needs to be bigger than window size."

    if window_len < 3:
        return x

    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"

    s = np.r_[2 * x[0] - x[window_len:1:-1],
              x, 2 * x[-1] - x[-1:-window_len:-1]]
    #print(len(s))

    if window == 'flat':  # moving average
        w = np.ones(window_len, 'd')
    else:
        w = getattr(np, window)(window_len)
    y = np.convolve(w / w.sum(), s, mode='same')
    return y[window_len - 1:-window_len + 1]

#Class to hold information about each frame


class Frame:
    def __init__(self, id, frame, value):
        self.id = id
        self.frame = frame
        self.value = value

    def __lt__(self, other):
        if self.id == other.id:
            return self.id < other.id
        return self.id < other.id

    def __gt__(self, other):
        return other.__lt__(self)

    def __eq__(self, other):
        return self.id == other.id and self.id == other.id

    def __ne__(self, other):
        return not self.__eq__(other)


def rel_change(a, b):
   x = (b - a) / max(a, b)
   print(x)
   return x


#print("Video :" + videopath)
#print("Frame Directory: " + dir)

def divideFrames(videopath,kf_dir,len_window):
    cap = cv2.VideoCapture(str(videopath))
    
    
    curr_frame = None
    prev_frame = None
    
    frame_diffs = []
    frames = []
    ret, frame = cap.read()
    i = 1
    
    while(ret):
        luv = cv2.cvtColor(frame, cv2.COLOR_BGR2LUV)
        curr_frame = luv
        if curr_frame is not None and prev_frame is not None:
            #logic here
            diff = cv2.absdiff(curr_frame, prev_frame)
            count = np.sum(diff)
            frame_diffs.append(count)
            frame = Frame(i, frame, count)
            frames.append(frame)
        prev_frame = curr_frame
        i = i + 1
        ret, frame = cap.read()
    """
        cv2.imshow('frame',luv)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    """
    cap.release()
    #cv2.destroyAllWindows()
    
    if USE_TOP_ORDER:
        # sort the list in descending order
        diff_array = np.array(frame_diffs)
        sm_diff_array = smooth(diff_array, len_window)
        frames.sort(key=operator.attrgetter("value"), reverse=True)
        for keyframe in frames[:NUM_TOP_FRAMES]:
            name = "frame_" + str(keyframe.id) + ".jpg"
            cv2.imwrite(kf_dir + "/" + name, keyframe.frame)
    
    if USE_THRESH:
        print("Using Threshold")
        for i in range(1, len(frames)):
            if (rel_change(np.float(frames[i - 1].value), np.float(frames[i].value)) >= THRESH):
                #print("prev_frame:"+str(frames[i-1].value)+"  curr_frame:"+str(frames[i].value))
                name = "frame_" + str(frames[i].id) + ".jpg"
                cv2.imwrite(kf_dir + "/" + name, frames[i].frame)
    
    
    if USE_LOCAL_MAXIMA:
        print("Using Local Maxima")
        diff_array = np.array(frame_diffs)
        sm_diff_array = smooth(diff_array, len_window)
        frame_indexes = np.asarray(argrelextrema(sm_diff_array, np.greater))[0]
        for i in frame_indexes:
            name = "frame_" + str(frames[i - 1].id) + ".jpg"
            #print(dir+name)
            cv2.imwrite(kf_dir +"/" + name, frames[i - 1].frame)
    
    
    plt.figure(figsize=(40, 20))
    plt.locator_params(numticks=100)
    plt.stem(sm_diff_array)
    plt.savefig(kf_dir +"/" + "/"+ 'plot.png')

In [ ]:
import os
import sys

# Directory where Videos are located
vidpath = "./Dataset/Videos"

# Directory where Keyframes are going to be stored
keypath = "./Dataset/Keyframes"

# smoothing window threshold for moving camera
CAM_MOV_THRESH = 18

for root, dirs, files in os.walk(vidpath):
    for cat in dirs:
        
        kdir =  keypath +"/" + cat
        
        if not os.path.exists(kdir):
            os.makedirs(kdir)
    
        for root,dirs,vids in os.walk(vidpath+"/"+cat): 
            
            for v in vids:
            
                vid_name = str(v).split(".")[0]
                kf_dir = kdir + "/" + vid_name
                
                if not os.path.exists(kf_dir):
                    os.makedirs(kf_dir)
                
                videopath = vidpath+"/"+cat+"/"+v
                                
                divideFrames(videopath,kf_dir,CAM_MOV_THRESH)

### Construct Object bank representation for each extracted keyframe 
Construct a new directory inside Dataset folder called Objectbank to store the objectbank representation for each video key frame.To construct object bank for each frame, we used the code given on Project website:
http://vision.stanford.edu/projects/objectbank/CPP_release.zip. 

Download and compile the C++ code to obtain executable `OBmain` which will be used in this python code.

Create another directory named Objectbank inside Dataset folder.

In [ ]:

import os
import subprocess as sp
import sys

# Directory where keyframes are located
rootkeyframes = "./Dataset/Keyframes/"

if not rootkeyframes.endswith('/'):
    rootkeyframes = rootkeyframes + "/"   

# Directory where object bank is created
rootobjbank = "./Dataset/Objectbank"

if not rootobjbank.endswith('/'):
    rootobjbank = rootobjbank + "/"

OB_BANK_GEN = "./OBmain"

cat_dirs = os.walk(rootkeyframes).next()[1]

for category in cat_dirs:
    
    #print("*****"+category+"****")
    
    # Make category directory in object bank root folder
    obj_cat_dir = rootobjbank + category
    
    if not os.path.exists(obj_cat_dir):
            os.makedirs(obj_cat_dir)
    
    
    kfdir_path = rootkeyframes + category
    keyframe_dirs = os.walk(kfdir_path).next()[1]
    
    for kfdirs in keyframe_dirs:
        
        # Make directory for each video in each category
        obj_kf_dir = obj_cat_dir + "/" + kfdirs
        if not os.path.exists(obj_kf_dir):
            os.makedirs(obj_kf_dir)
        
        
        kf_path = kfdir_path + "/"+kfdirs
        print(kf_path)
        
        if os.path.exists(kf_path+"/plot.png"):
            os.remove(kf_path+"/plot.png")
        
        inputdirectory = os.path.abspath(kf_path) + "/"
        outputdirectory = os.path.abspath(obj_kf_dir ) + "/"
        
        
        sp.call([OB_BANK_GEN , inputdirectory , outputdirectory])

./Dataset/Keyframes/Badminton/_5QkmjT7v9k


### Construct VGG16 feature bank representation for each extracted keyframe 
Construct a new directory inside Dataset folder called VGGfeaturebank to store the feature representation for each video key frame.To construct feature bank for each frame, we used the pre-trained VGG16 network available in keras library.

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras import backend 
import os
import sys
import glob
import numpy as np

# load the model
model = VGG16()


rootkeyframes = "./Dataset/Keyframes/"

if not rootkeyframes.endswith('/'):
    rootkeyframes = rootkeyframes + "/"   

# Directory where vgg object bank is created
rootvggobjbank = "./Dataset/VGGbank"

if not rootvggobjbank.endswith('/'):
    rootkeyframes = rootkeyframes + "/"


cat_dirs = os.walk(rootkeyframes).next()[1]

for category in cat_dirs:
    
    # Make category directory in object bank root folder
    obj_cat_dir = rootvggobjbank + category
    
    if not os.path.exists(obj_cat_dir):
            os.makedirs(obj_cat_dir)
    
    
    kfdir_path = rootkeyframes + category
    keyframe_dirs = os.walk(kfdir_path).next()[1]
    
    for kfdirs in keyframe_dirs:
        
        # Make directory for each video in each category
        obj_kf_dir = obj_cat_dir + "/" + kfdirs
        if not os.path.exists(obj_kf_dir):
            os.makedirs(obj_kf_dir)
        
        
        kf_path = kfdir_path + "/"+kfdirs
        #print(kf_path)
        
        if os.path.exists(kf_path+"/plot.png"):
            os.remove(kf_path+"/plot.png")
        
        inputdirectory = os.path.abspath(kf_path) + "/"
        outputdirectory = os.path.abspath(obj_kf_dir ) + "/"
        
        
        for fname in glob.glob(inputdirectory + "/*.jpg"):
            
            img_name = fname.split('.')[0].split('/')[-1] 
                       
            # load an image from file
            image = load_img(fname, target_size=(224, 224))
            # convert the image pixels to a numpy array
            image = img_to_array(image)
            # reshape data for the model
            image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            # prepare the image for the VGG model
            image = preprocess_input(image)
            # predict the probability across all output classes
            yhat = model.predict(image)
            # convert the probabilities to class labels
            label = decode_predictions(yhat)
            # with a Sequential model
            get_2ndlast_layer_output = backend.function([model.layers[0].input],
                                  [model.layers[-2].output])
            layer_output = get_2ndlast_layer_output([image])[0]
            
            out_file_name = outputdirectory + img_name + "_vgg.feat"
            
            np.savetxt(out_file_name, layer_output)
            #print("Generated "+ out_file_name)